In [1]:
import glob
import json
import numpy as np
from utils.analysis import Extrapolation

In [2]:
TAG = "main"
babies = glob.glob(f"../analysis/studies/vbswh/output_{TAG}/Run2/*.root")
babies = [baby for baby in babies if "Lambda" not in baby and "VBSWH_SM" not in baby]
data_babies = [baby for baby in babies if "data" in baby]
sig_babies = [baby for baby in babies if "VBSWH_mkW" in baby]
bkg_babies = list(set(babies) - set(data_babies + sig_babies))
print(data_babies)
print(sig_babies)
bkg_babies

['../analysis/studies/vbswh/output_main/Run2/data.root']
['../analysis/studies/vbswh/output_main/Run2/VBSWH_mkW.root']


['../analysis/studies/vbswh/output_main/Run2/EWKWLep.root',
 '../analysis/studies/vbswh/output_main/Run2/TTbar2L.root',
 '../analysis/studies/vbswh/output_main/Run2/Bosons.root',
 '../analysis/studies/vbswh/output_main/Run2/VH.root',
 '../analysis/studies/vbswh/output_main/Run2/TTX.root',
 '../analysis/studies/vbswh/output_main/Run2/WJets.root',
 '../analysis/studies/vbswh/output_main/Run2/TTbar1L.root',
 '../analysis/studies/vbswh/output_main/Run2/SingleTop.root']

In [3]:
# bkg_babies = [b for b in bkg_babies if "TTbar1L" in b or "TTbar2L" in b]
# bkg_babies = [b for b in bkg_babies if not ("TTbar1L" in b or "TTbar2L" in b)]
# bkg_babies = [b for b in bkg_babies if "TTbar1L" in b]
# bkg_babies = [b for b in bkg_babies if "TTbar2L" in b]
bkg_babies

['../analysis/studies/vbswh/output_main/Run2/EWKWLep.root',
 '../analysis/studies/vbswh/output_main/Run2/TTbar2L.root',
 '../analysis/studies/vbswh/output_main/Run2/Bosons.root',
 '../analysis/studies/vbswh/output_main/Run2/VH.root',
 '../analysis/studies/vbswh/output_main/Run2/TTX.root',
 '../analysis/studies/vbswh/output_main/Run2/WJets.root',
 '../analysis/studies/vbswh/output_main/Run2/TTbar1L.root',
 '../analysis/studies/vbswh/output_main/Run2/SingleTop.root']

In [4]:
vbswh = Extrapolation(
    sig_root_files=sig_babies,
    bkg_root_files=bkg_babies,
    data_root_files=data_babies,
    ttree_name="tree",
    weight_columns=[
        "xsec_sf", "lep_id_sf", "ewkfix_sf", 
        "elec_reco_sf", "muon_iso_sf", 
        "btag_sf", "pu_sf", "prefire_sf", "trig_sf", "puid_sf",
        "xbb_sf" # applied only because Xbb > 0.9 applied everywhere for ABCD
    ],
    reweight_column="reweights",
    plots_dir=f"/home/users/jguiang/public_html/onelep_plots/{TAG}/val"
)
vbswh.df["presel_noVBS_noBVeto"] = vbswh.df.eval(
    "hbbjet_score > 0.3"
)
vbswh.df["presel_noVBS"] = vbswh.df.eval(
    "passes_bveto and hbbjet_score > 0.3"
)
vbswh.df["presel_noDetaJJ"] = vbswh.df.eval(
    "passes_bveto and M_jj > 500 and hbbjet_score > 0.3"
)
vbswh.df["presel"] = vbswh.df.eval(
    "passes_bveto and M_jj > 500 and abs(deta_jj) > 3 and hbbjet_score > 0.3"
)

ORIG_EVENT_WEIGHT = vbswh.df.event_weight.values.copy()

Loading sig babies:   0%|                                                                                                                                   | 0/1 [00:00<?, ?it/s]/home/users/jguiang/projects/vbs/notebooks/utils/analysis.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["is_signal"] = True
/home/users/jguiang/projects/vbs/notebooks/utils/analysis.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["is_data"] = False
Loading bkg babies:   0%|                                                                

Loading data babies:   0%|                                                                                                                                  | 0/1 [00:00<?, ?it/s]/home/users/jguiang/projects/vbs/notebooks/utils/analysis.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["is_signal"] = False
/home/users/jguiang/projects/vbs/notebooks/utils/analysis.py:71: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["is_data"] = True
Loading data babies: 100%|███████████████████████████████████████████████████████████████

In [5]:
def get_year_str(year, doAPV=True):
    if doAPV and year == -2016:
        return "2016preVFP"
    elif doAPV and year == 2016:
        return "2016postVFP"
    else:
        return str(year)

In [6]:
def ABCD_correction(A, B, C, D, sample=None, blind=True):
    
    A_sig_count = vbswh.sig_count(selection=A)
    B_sig_count = vbswh.sig_count(selection=B)
    C_sig_count = vbswh.sig_count(selection=C)
    D_sig_count = vbswh.sig_count(selection=D)

    A_sig_error = vbswh.sig_error(selection=A)
    B_sig_error = vbswh.sig_error(selection=B)
    C_sig_error = vbswh.sig_error(selection=C)
    D_sig_error = vbswh.sig_error(selection=D)
    
    A_data_count = vbswh.data_count(selection=A)
    B_data_count = vbswh.data_count(selection=B)
    C_data_count = vbswh.data_count(selection=C)
    D_data_count = vbswh.data_count(selection=D)
        
    if sample is None:
        A_bkg_count = vbswh.bkg_count(selection=A)
        B_bkg_count = vbswh.bkg_count(selection=B)
        C_bkg_count = vbswh.bkg_count(selection=C)
        D_bkg_count = vbswh.bkg_count(selection=D)
        
        A_bkg_error = vbswh.bkg_error(selection=A)
        B_bkg_error = vbswh.bkg_error(selection=B)
        C_bkg_error = vbswh.bkg_error(selection=C)
        D_bkg_error = vbswh.bkg_error(selection=D)
    else:
        A_bkg_count = vbswh.sample_count(sample, selection=A)
        B_bkg_count = vbswh.sample_count(sample, selection=B)
        C_bkg_count = vbswh.sample_count(sample, selection=C)
        D_bkg_count = vbswh.sample_count(sample, selection=D)
        
        A_bkg_error = vbswh.sample_error(sample, selection=A)
        B_bkg_error = vbswh.sample_error(sample, selection=B)
        C_bkg_error = vbswh.sample_error(sample, selection=C)
        D_bkg_error = vbswh.sample_error(sample, selection=D)
        
        A_data_count -= (vbswh.bkg_count(selection=A) - A_bkg_count)
        B_data_count -= (vbswh.bkg_count(selection=B) - B_bkg_count)
        C_data_count -= (vbswh.bkg_count(selection=C) - C_bkg_count)
        D_data_count -= (vbswh.bkg_count(selection=D) - D_bkg_count)
        
    A_data_error = np.sqrt(A_data_count)
    B_data_error = np.sqrt(B_data_count)
    C_data_error = np.sqrt(C_data_count)
    D_data_error = np.sqrt(D_data_count)
    
    if blind:
        D_data_count = 0
        D_data_error = 0
        
    print("region,bkg_count,bkg_error,sig_count,sig_error,data_count,data_err")
    print(f"A,{A_bkg_count:.2f},{A_bkg_error:.2f},{A_sig_count:.2f},{A_bkg_error:.2f},{A_data_count:.2f},{A_data_error:.2f}")
    print(f"B,{B_bkg_count:.2f},{B_bkg_error:.2f},{B_sig_count:.2f},{B_bkg_error:.2f},{B_data_count:.2f},{B_data_error:.2f}")
    print(f"C,{C_bkg_count:.2f},{C_bkg_error:.2f},{C_sig_count:.2f},{C_bkg_error:.2f},{C_data_count:.2f},{C_data_error:.2f}")
    print(f"D,{D_bkg_count:.2f},{D_bkg_error:.2f},{D_sig_count:.2f},{D_bkg_error:.2f},{D_data_count:.2f},{D_data_error:.2f}")
        
    # Transfer factor
    mc_tf = A_bkg_count/B_bkg_count
    mc_tf_err = mc_tf*np.sqrt((A_bkg_error/A_bkg_count)**2 + (B_bkg_error/B_bkg_count)**2)
    
    data_tf = A_data_count/B_data_count
    data_tf_err = data_tf*np.sqrt((A_data_error/A_data_count)**2 + (B_data_error/B_data_count)**2)
    
    # Prediction
    D_bkg_pred = mc_tf*C_bkg_count
    syst = abs(1 - D_bkg_pred/D_bkg_count)
    print(syst)
    
    # Transfer factor correction
    mc_tf_fix = (D_bkg_count/C_bkg_count)/mc_tf
    print(f"Calculated MC TF fix: {mc_tf_fix}")
    
    mc_tf *= mc_tf_fix
    data_tf *= mc_tf_fix
    
    # Prediction
    D_bkg_pred = mc_tf*C_bkg_count
    D_bkg_pred_err = D_bkg_pred*np.sqrt(
        (A_bkg_error/A_bkg_count)**2 + (B_bkg_error/B_bkg_count)**2 + (C_bkg_error/C_bkg_count)**2
    )
    
    D_data_pred = data_tf*C_data_count
    D_data_pred_err = D_data_pred*np.sqrt(
        (A_data_error/A_data_count)**2 + (B_data_error/B_data_count)**2 + (C_data_error/C_data_count)**2
    )
    
    print(f"D_pred,{D_bkg_pred:.2f},{D_bkg_pred_err:.2f},,,{D_data_pred:.2f},{D_data_pred_err:.2f}")
    
    return D_data_pred, D_data_pred_err, syst*D_data_pred

In [7]:
doAPV = True
years = [2016, 2017, 2018] if not doAPV else [-2016, 2016, 2017, 2018]
for year in years:
    print()
    print(f"Running {year}")
    
    year_str = get_year_str(year, doAPV=doAPV)
    year_check = f"abs(year) == {year}" if not doAPV else f"year == {year}"
    SRlike  = f"{year_check} and presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9"
    regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150"
    regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150"
    regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop < 150"
    regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"
    AN_numbers = {
        "PredBkg": 0,
        "PredBkgStatErr": 0,
        "PredBkgSystErr": 0,
        "ExpSig": round(vbswh.sig_count(selection=regionD)),
        "ExpSigStatErr": round(vbswh.sig_error(selection=regionD), 1),
        "ExpSigSystErr": 0,
        "ExpBkg": round(vbswh.bkg_count(selection=regionD)),
        "BkgEstABMC": 0,
        "BkgEstABMCErr": 0,
        "BkgEstABData": 0,
        "BkgEstABDataErr": 0,
        "PredBkgMC": 0,
        "BkgEstMethodSystErr": 0,
        "BkgEstBkgCompSystErr": 0,
        "BkgEstTotalSystErr": 0,
        "BkgEstStatErr": 0,
        "BkgEstWJetsUpABMC": 0,
        "BkgEstWJetsUpABMCErr": 0,
        "BkgEstWJetsDownABMC": 0,
        "BkgEstWJetsDownABMCErr": 0,
        "BkgEstWJetsCompSyst": 0,
        "BkgEstBosonsUpABMC": 0,
        "BkgEstBosonsUpABMCErr": 0,
        "BkgEstBosonsDownABMC": 0,
        "BkgEstBosonsDownABMCErr": 0,
        "BkgEstBosonsCompSyst": 0,
    }


    # Regular ABCD
    pred, stat, syst = vbswh.ABCD( 
        "abs(deta_jj) > 4",
        "hbbjet_msoftdrop < 150",
        SRlike,
        h_dir="left", v_dir="up", 
        show_data=True
    )
#     AN_numbers["PredBkg"] = round(pred)
#     AN_numbers["PredBkgStatErr"] = round(pred*stat, 1)
    A_bkg_wgt = vbswh.bkg_count(selection=regionA)
    A_bkg_err = vbswh.bkg_error(selection=regionA)
    B_bkg_wgt = vbswh.bkg_count(selection=regionB)
    B_bkg_err = vbswh.bkg_error(selection=regionB)
    AN_numbers["PredBkgMC"] = round(A_bkg_wgt/B_bkg_wgt*vbswh.bkg_count(selection=regionC), 1)
    AN_numbers["BkgEstABMC"] = A_bkg_wgt/B_bkg_wgt
    AN_numbers["BkgEstABMCErr"] = round(np.sqrt((B_bkg_err/B_bkg_wgt)**2 + (A_bkg_err/A_bkg_wgt)**2)*100, 1)
    AN_numbers["BkgEstMethodSystErr"] = syst*100
    AN_numbers["BkgEstStatErr"] = stat*100
    A_data     = vbswh.data_count(selection=regionA)
    A_data_err = vbswh.data_error(selection=regionA)
    B_data     = vbswh.data_count(selection=regionB)
    B_data_err = vbswh.data_error(selection=regionB)
    AN_numbers["BkgEstABData"] = A_data/B_data
    AN_numbers["BkgEstABDataErr"] = round(np.sqrt((B_data_err/B_data)**2 + (A_data_err/A_data)**2)*100, 1)


    # WJets up by factor of 2
    vbswh.df.loc[vbswh.df.name == "WJets", "event_weight"] *= 2
    vbswh.ABCD( 
        "abs(deta_jj) > 4",
        "hbbjet_msoftdrop < 150",
        SRlike,
        h_dir="left", v_dir="up", 
        show_data=True
    )
    A_bkg_wgt = vbswh.bkg_count(selection=regionA)
    A_bkg_err = vbswh.bkg_error(selection=regionA)
    B_bkg_wgt = vbswh.bkg_count(selection=regionB)
    B_bkg_err = vbswh.bkg_error(selection=regionB)
    AN_numbers["BkgEstWJetsUpABMC"] = A_bkg_wgt/B_bkg_wgt
    AN_numbers["BkgEstWJetsUpABMCErr"] = round(np.sqrt((B_bkg_err/B_bkg_wgt)**2 + (A_bkg_err/A_bkg_wgt)**2)*100, 1)
    # Reset
    vbswh.df.event_weight = ORIG_EVENT_WEIGHT.copy()
    print("")
    # WJets down by factor of 2
    vbswh.df.loc[vbswh.df.name == "WJets", "event_weight"] *= 0.5
    vbswh.ABCD( 
        "abs(deta_jj) > 4",
        "hbbjet_msoftdrop < 150",
        SRlike,
        h_dir="left", v_dir="up", 
        show_data=True
    )
    A_bkg_wgt = vbswh.bkg_count(selection=regionA)
    A_bkg_err = vbswh.bkg_error(selection=regionA)
    B_bkg_wgt = vbswh.bkg_count(selection=regionB)
    B_bkg_err = vbswh.bkg_error(selection=regionB)
    AN_numbers["BkgEstWJetsDownABMC"] = A_bkg_wgt/B_bkg_wgt
    AN_numbers["BkgEstWJetsDownABMCErr"] = round(np.sqrt((B_bkg_err/B_bkg_wgt)**2 + (A_bkg_err/A_bkg_wgt)**2)*100, 1)
    AN_numbers["BkgEstWJetsCompSyst"] = 100*max(
        abs(1 - AN_numbers["BkgEstWJetsUpABMC"]/AN_numbers["BkgEstABMC"]),
        abs(1 - AN_numbers["BkgEstWJetsDownABMC"]/AN_numbers["BkgEstABMC"])
    )
    # Reset
    vbswh.df.event_weight = ORIG_EVENT_WEIGHT.copy()


    # Bosons up by factor of 2
    vbswh.df.loc[vbswh.df.name == "Bosons", "event_weight"] *= 2
    vbswh.ABCD( 
        "abs(deta_jj) > 4",
        "hbbjet_msoftdrop < 150",
        SRlike,
        h_dir="left", v_dir="up", 
        show_data=True
    )
    A_bkg_wgt = vbswh.bkg_count(selection=regionA)
    A_bkg_err = vbswh.bkg_error(selection=regionA)
    B_bkg_wgt = vbswh.bkg_count(selection=regionB)
    B_bkg_err = vbswh.bkg_error(selection=regionB)
    AN_numbers["BkgEstBosonsUpABMC"] = A_bkg_wgt/B_bkg_wgt
    AN_numbers["BkgEstBosonsUpABMCErr"] = round(np.sqrt((B_bkg_err/B_bkg_wgt)**2 + (A_bkg_err/A_bkg_wgt)**2)*100, 1)
    # Reset
    vbswh.df.event_weight = ORIG_EVENT_WEIGHT.copy()
    print("")
    # Bosons down by factor of 2
    vbswh.df.loc[vbswh.df.name == "Bosons", "event_weight"] *= 0.5
    vbswh.ABCD( 
        "abs(deta_jj) > 4",
        "hbbjet_msoftdrop < 150",
        SRlike,
        h_dir="left", v_dir="up", 
        show_data=True
    )
    A_bkg_wgt = vbswh.bkg_count(selection=regionA)
    A_bkg_err = vbswh.bkg_error(selection=regionA)
    B_bkg_wgt = vbswh.bkg_count(selection=regionB)
    B_bkg_err = vbswh.bkg_error(selection=regionB)
    AN_numbers["BkgEstBosonsDownABMC"] = A_bkg_wgt/B_bkg_wgt
    AN_numbers["BkgEstBosonsDownABMCErr"] = round(np.sqrt((B_bkg_err/B_bkg_wgt)**2 + (A_bkg_err/A_bkg_wgt)**2)*100, 1)
    AN_numbers["BkgEstBosonsCompSyst"] = 100*max(
        abs(1 - AN_numbers["BkgEstBosonsUpABMC"]/AN_numbers["BkgEstABMC"]),
        abs(1 - AN_numbers["BkgEstBosonsDownABMC"]/AN_numbers["BkgEstABMC"])
    )
    # Reset
    vbswh.df.event_weight = ORIG_EVENT_WEIGHT.copy()



    AN_numbers["BkgEstBkgCompSystErr"] = np.sqrt(
        AN_numbers["BkgEstWJetsCompSyst"]**2 + AN_numbers["BkgEstBosonsCompSyst"]**2
    )
    AN_numbers["BkgEstTotalSystErr"] = np.sqrt(
        AN_numbers["BkgEstMethodSystErr"]**2 + AN_numbers["BkgEstBkgCompSystErr"]**2
    )

    
    D_pred, D_pred_stat_err, _ = ABCD_correction(regionA, regionB, regionC, regionD)
    AN_numbers["PredBkg"] = round(D_pred)
    AN_numbers["PredBkgStatErr"] = round(D_pred_stat_err, 1)


    AN_numbers["PredBkgSystErr"] = round(D_pred*AN_numbers["BkgEstTotalSystErr"]/100, 1)

    AN_numbers["BkgEstABMC"] = round(AN_numbers["BkgEstABMC"], 2)
    AN_numbers["BkgEstABData"] = round(AN_numbers["BkgEstABData"], 2)

    AN_numbers["BkgEstWJetsUpABMC"] = round(AN_numbers["BkgEstWJetsUpABMC"], 2)
    AN_numbers["BkgEstWJetsDownABMC"] = round(AN_numbers["BkgEstWJetsDownABMC"], 2)
    AN_numbers["BkgEstWJetsCompSyst"] = round(AN_numbers["BkgEstWJetsCompSyst"], 1)

    AN_numbers["BkgEstBosonsUpABMC"] = round(AN_numbers["BkgEstBosonsUpABMC"], 2)
    AN_numbers["BkgEstBosonsDownABMC"] = round(AN_numbers["BkgEstBosonsDownABMC"], 2)
    AN_numbers["BkgEstBosonsCompSyst"] = round(AN_numbers["BkgEstBosonsCompSyst"], 1)

    AN_numbers["BkgEstBkgCompSystErr"] = round(AN_numbers["BkgEstBkgCompSystErr"], 1)
    AN_numbers["BkgEstMethodSystErr"] = round(AN_numbers["BkgEstMethodSystErr"], 1)
    AN_numbers["BkgEstTotalSystErr"] = round(AN_numbers["BkgEstTotalSystErr"], 1)
    AN_numbers["BkgEstStatErr"] = round(AN_numbers["BkgEstStatErr"], 1)

    with open(f"AN_numbers_{year_str}.json", "w") as f_out:
        json.dump(AN_numbers, f_out)


Running -2016
cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
year == -2016 and presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,27.726372639355873,1.3263454959240522,1.1223262546600545,0.10417589173250218,17,4.123105625617661
year == -2016 and presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,34.38577592330003,1.3001397350612207,0.04957513752908121,0.022267885712137697,19,4.358898943540674
year == -2016 and presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,27.63366194258552,1.671486601073621,1.422647661879005,0.11585191899035299,18,4.242640687119285
year == -2016 and presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150,D,15.293271506471914,1.1575345623904052,49.329888457709124,0.69275


cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
year == 2016 and presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,21.951452618830054,1.148099459939339,0.8447690397199017,0.0943301000925796,18,4.242640687119285
year == 2016 and presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,30.98274005512387,1.2906915979616995,0.03205996790883912,0.01795985088000894,37,6.082762530298219
year == 2016 and presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,19.834043796824993,1.4274348932128498,1.3296037323218943,0.11320749184417198,12,3.4641016151377544
year == 2016 and presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150,D,11.548711895173312,1.1150184668725092,42.94167884675472,0.6438799327868056,BLINDE

region,bkg_count,bkg_error,sig_count,sig_error,data_count,data_err
A,44.97,1.54,1.84,1.54,43.00,6.56
B,65.69,3.04,0.02,3.04,68.00,8.25
C,49.78,2.17,3.24,2.17,57.00,7.55
D,33.56,2.20,104.02,2.20,0.00,0.00
0.015447447409018933
Calculated MC TF fix: 0.9847875461714596
D_pred,33.56,2.42,,,35.50,8.36

Running 2018
cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
year == 2018 and presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,77.63239851046919,2.252596795188746,3.112231316637598,0.32005230367735066,64,8.0
year == 2018 and presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,109.14276113750454,4.617139645328312,0.17135803904224803,0.07680163384677141,77,8.774964387392123
year == 2018 and presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,80.8720741368505,3.

In [8]:
for year in years:
    year_str = get_year_str(year, doAPV=doAPV)
    with open(f"AN_numbers_{year_str}.json", "r") as f_in:
        nums = json.load(f_in)
        print(f"{year_str}: {nums['PredBkg']} +- {nums['PredBkgSystErr']} +- {nums['PredBkgStatErr']}")
        print(f"{year_str}: {nums['BkgEstTotalSystErr']}% = sqrt({nums['BkgEstMethodSystErr']}%^2 + {nums['BkgEstBkgCompSystErr']}%^2)")
        print(f"{year_str}: {nums['BkgEstStatErr']}%")
        print()
        
# BkgEstBkgCompSystErr
# BkgEstTotalSystErr
# BkgEstStatErr

2016preVFP: 11 +- 5.1 +- 4.5
2016preVFP: 46.3% = sqrt(45.7%^2 + 7.6%^2)
2016preVFP: 40.9%

2016postVFP: 5 +- 1.1 +- 1.9
2016postVFP: 24.3% = sqrt(24.0%^2 + 4.1%^2)
2016postVFP: 40.7%

2017: 35 +- 2.2 +- 8.4
2017: 6.3% = sqrt(1.5%^2 + 6.1%^2)
2017: 23.6%

2018: 66 +- 5.3 +- 13.3
2018: 8.1% = sqrt(4.9%^2 + 6.4%^2)
2018: 20.2%



In [9]:
for year_i, year in enumerate(years):
    year_str = get_year_str(year, doAPV=doAPV)
    with open(f"AN_numbers_{year_str}.json", "r") as f_in:
        nums = json.load(f_in)
        for key in ["PredBkg", "PredBkgStatErr", "PredBkgSystErr"]:
            value = nums[key]
            if year_i == 0:
                key += "YearOne"
            elif year_i == 1:
                key += "YearTwo"
            elif year_i == 2:
                key += "YearThree"
            elif year_i == 3:
                key += "YearFour"
            print(f"\\newcommand{{\\{key}}}{{{value}}}")

\newcommand{\PredBkgYearOne}{11}
\newcommand{\PredBkgStatErrYearOne}{4.5}
\newcommand{\PredBkgSystErrYearOne}{5.1}
\newcommand{\PredBkgYearTwo}{5}
\newcommand{\PredBkgStatErrYearTwo}{1.9}
\newcommand{\PredBkgSystErrYearTwo}{1.1}
\newcommand{\PredBkgYearThree}{35}
\newcommand{\PredBkgStatErrYearThree}{8.4}
\newcommand{\PredBkgSystErrYearThree}{2.2}
\newcommand{\PredBkgYearFour}{66}
\newcommand{\PredBkgStatErrYearFour}{13.3}
\newcommand{\PredBkgSystErrYearFour}{5.3}
